# 小编环境

In [1]:
import sys

print('python 版本：',sys.version.split('|')[0])
#python 版本： 3.11.9 

import polars as pl

print("polars 版本：",pl.__version__)
#polars 版本： 0.20.22

python 版本： 3.11.9 
polars 版本： 0.20.22


# Lazy / Eager API 区别

- Eager API：
实时进行计算，每一步操作都会进行计算，类似pandas那样，每操作一步都会进行计算，得到这一步的结果，所见即所得，如果没有明确指定或者调用特定的方法之外，polars 基本都是使用该模式
    

- Lazy API：
推迟进行计算，把所有的操作步骤先记下来，Query plan（查询计划），等到需要结果时，才统一进行计算，polars 会对这些计算步骤自动进行优化，提升性能
    - pl.scan_csv 等 pl.scan_ 函数
    - 调用DataFrame 的 .lazy 方法，转换为 Lazy 模式

# Eager API 数据处理案例

In [2]:
df = pl.read_csv("./data/iris.csv")
df_small = df.filter(pl.col("Sepal.Length") > 5)
df_agg = df_small.group_by("Species").agg(pl.col("Sepal.Width").mean())
print(df_agg)

shape: (3, 2)
┌────────────┬─────────────┐
│ Species    ┆ Sepal.Width │
│ ---        ┆ ---         │
│ str        ┆ f64         │
╞════════════╪═════════════╡
│ versicolor ┆ 2.804255    │
│ setosa     ┆ 3.713636    │
│ virginica  ┆ 2.983673    │
└────────────┴─────────────┘


# Lazy API 数据处理案例

In [3]:
q = (
    pl.scan_csv("./data/iris.csv")
    .filter(pl.col("Sepal.Length") > 5)
    .group_by("Species")
    .agg(pl.col("Sepal.Width").mean())
)

In [4]:
type(q)

polars.lazyframe.frame.LazyFrame

In [5]:
q

In [6]:
df = q.collect()

print(df)

shape: (3, 2)
┌────────────┬─────────────┐
│ Species    ┆ Sepal.Width │
│ ---        ┆ ---         │
│ str        ┆ f64         │
╞════════════╪═════════════╡
│ versicolor ┆ 2.804255    │
│ virginica  ┆ 2.983673    │
│ setosa     ┆ 3.713636    │
└────────────┴─────────────┘


# 使用建议

- 如果你是在进行探索性分析，想知道中间的每个步骤数据情况，那么可以使用 Eager 模式
- 如果想得到最终的计算结果，那么可以使用 Lazy 模式，让polars对中间的计算进行优化，提升数据处理效率

注：在大部分情况下，Eager API 背后其实调用的是 Lazy API，这样Eager 模式也可以利用上查询优化
